In [25]:
import os  # For directory traversal
import pandas as pd  # For working with DataFrames
import numpy as np

events = [0, 1]  # Specify the event numbers, start from 0
event_dates = [
    "2024-11-19 19:30:00",
    "2024-11-29 19:00:00",
    "2024-10-19 19:30:00",
    "2025-02-27 19:30:00",
    "2024-11-21 19:30:00",
    "2024-10-22 19:00:00",
    "2024-11-13 19:00:00",
    "2025-01-23 19:00:00",
    "2024-10-25 19:00:00",
    "2024-11-23 19:30:00",
    "2024-11-10 18:30:00",
    "2024-10-26 19:30:00",
    "2024-11-08 19:00:00"
]

df = pd.DataFrame()

# Iterate through event directories
for event, event_dir in enumerate(os.listdir('data')):
    if event in events:
        for root, dir, files in os.walk('data/'+event_dir):
            if not files:
                continue
            f = files[0]
            if f.endswith('.csv'):
                csv_path = os.path.join(root, f)
                temp_df = pd.read_csv(csv_path)
                # temp_df['min_option'] = temp_df['quantity_options'].apply(min).apply(pd.to_numeric, errors='coerce')
                temp_df['event'] = event  # Add the 'event' column to identify the source event
                temp_df['price'] = temp_df['price_per_ticket']

                # columns to keep
                temp_df = temp_df[['section', 'row', 'seat_start', 'seat_end', 'quantity', 'price', 'update_time', 'event']]
                df = pd.concat([df, temp_df], ignore_index=True)

df['update_time'] = pd.to_datetime(df['update_time'])

df['seat_start'] = df['seat_start'].apply(pd.to_numeric, errors='coerce')
df['seat_end'] = df['seat_end'].apply(pd.to_numeric, errors='coerce')
df = df.dropna()
df['seat_start'] = df['seat_start'].astype(int)
df['seat_end'] = df['seat_end'].astype(int)

df = df.dropna()
df

,section,row,seat_start,seat_end,quantity,price,update_time,event
0,216,8,11,13,3,511.35,2024-09-05 13:13:49+00:00,1
1,216,8,14,15,2,356.54,2024-09-28 00:32:13+00:00,1
2,216,8,16,17,2,433.26,2024-09-08 17:45:20+00:00,1
3,216,8,3,4,2,403.12,2024-09-26 05:27:54+00:00,1
4,216,8,11,13,3,511.35,2024-09-05 13:13:49+00:00,1
...,...,...,...,...,...,...,...,...
627937,105,5,6,7,2,434.63,2024-10-25 04:29:11+00:00,1
627938,105,5,6,7,2,434.63,2024-10-25 04:29:11+00:00,1
627939,118,6,6,9,4,497.65,2024-10-25 19:29:09+00:00,1
627940,118,6,6,9,4,497.65,2024-10-25 19:29:09+00:00,1


1. Unpacking seats.
2. Creating Event-Section-Row-Seat dataframe "srs" and removing duplicates considering all columns excluding time and quantity ().

In [26]:
expanded_rows = []
for _, row in df.iterrows():
    row_dict = row.to_dict()  # Convert the row to a dictionary
    for i in range(row['seat_start'], row['seat_end'] + 1):
        new_row = row_dict.copy()  # Copy the original row's data
        new_row['seat'] = i  # Update the seat value
        expanded_rows.append(new_row)  # Add the new row to the list

srs = pd.DataFrame(expanded_rows).drop(columns=['seat_start', 'seat_end'])
srs = srs.drop_duplicates()
srs = srs.reset_index().set_index(['event', 'section', 'row', 'seat'])
srs

index  quantity   price               update_time
event section row seat                                                     
1     216     8   11          0         3  511.35 2024-09-05 13:13:49+00:00
                  12          1         3  511.35 2024-09-05 13:13:49+00:00
                  13          2         3  511.35 2024-09-05 13:13:49+00:00
                  14          3         2  356.54 2024-09-28 00:32:13+00:00
                  15          4         2  356.54 2024-09-28 00:32:13+00:00
...                         ...       ...     ...                       ...
      105     5   7     1565377         2  434.63 2024-10-25 04:29:11+00:00
      118     6   6     1565406         4  497.65 2024-10-25 19:29:09+00:00
                  7     1565407         4  497.65 2024-10-25 19:29:09+00:00
                  8     1565408         4  497.65 2024-10-25 19:29:09+00:00
                  9     1565409         4  497.65 2024-10-25 19:29:09+00:00

[38838 rows x 4 columns]

3. Time to Days (before event).

In [27]:
srs['t'] = srs.index.map(lambda idx: pd.to_datetime(event_dates[idx[0]]).tz_localize('UTC'))  # event time
srs['days'] = (srs['t'] - pd.to_datetime(srs['update_time'])).dt.total_seconds() / 86400
srs = srs[['quantity', 'price', 'days']]
srs

quantity   price       days
event section row seat                             
1     216     8   11           3  511.35  85.240405
                  12           3  511.35  85.240405
                  13           3  511.35  85.240405
                  14           2  356.54  62.769294
                  15           2  356.54  62.769294
...                          ...     ...        ...
      105     5   7            2  434.63  35.604734
      118     6   6            4  497.65  34.979757
                  7            4  497.65  34.979757
                  8            4  497.65  34.979757
                  9            4  497.65  34.979757

[38838 rows x 3 columns]

### FIRST MAIN OUTPUT: all prices per Event-Section-Row-Seat-Quantity-Time points

In [28]:
srs.to_csv('preprocessed.csv')
srs

quantity   price       days
event section row seat                             
1     216     8   11           3  511.35  85.240405
                  12           3  511.35  85.240405
                  13           3  511.35  85.240405
                  14           2  356.54  62.769294
                  15           2  356.54  62.769294
...                          ...     ...        ...
      105     5   7            2  434.63  35.604734
      118     6   6            4  497.65  34.979757
                  7            4  497.65  34.979757
                  8            4  497.65  34.979757
                  9            4  497.65  34.979757

[38838 rows x 3 columns]

### Generalization for time. Creation of particular time price (or produced from time-related data price)

4. We are getting rid of time!
5. IMPORTANT. OPTIONAL. We can investigate time related connections here and store our findings to use it for a separate model.

In [29]:
srsq = srs.set_index('quantity', append=True)
srsq = srsq.groupby(srsq.index.names).agg({
    'days': list,
    'price': list
})
srsq

days  \
event section row seat quantity                                                      
1     101     1   1    4         [48.81136574074074, 48.81136574074074, 48.8113...   
                  2    4         [48.81136574074074, 48.81136574074074, 48.8113...   
                  3    4         [48.81136574074074, 48.81136574074074, 48.8113...   
                  4    4         [48.81136574074074, 48.81136574074074, 48.8113...   
                  5    2         [58.88381944444444, 51.88789351851852, 44.8671...   
...                                                                            ...   
      UCTER   2   59   4                   [58.21869212962963, 57.691747685185184]   
                  60   4                   [58.21869212962963, 57.691747685185184]   
                  61   4                   [58.21869212962963, 57.691747685185184]   
                  6292 2         [55.05679398148148, 54.26576388888889, 51.2058...   
                  6293 2         [55.05679398148148, 54.26576388888889, 51.2058...   

                                                                             price  
event section row seat quantity                                                     
1     101     1   1    4                       [1915.6, 1758.05, 1442.95, 1207.31]  
                  2    4                       [1915.6, 1758.05, 1442.95, 1207.31]  
                  3    4                       [1915.6, 1758.05, 1442.95, 1207.31]  
                  4    4                       [1915.6, 1758.05, 1442.95, 1207.31]  
                  5    2         [3477.4, 2758.15, 1895.05, 1751.2, 1536.11, 14...  
...                                                                            ...  
      UCTER   2   59   4                                          [229.13, 222.28]  
                  60   4                                          [229.13, 222.28]  
                  61   4                                          [229.13, 222.28]  
                  6292 2                               [1842.99, 1842.99, 1842.99]  
                  6293 2                               [1842.99, 1842.99, 1842.99]  

[6906 rows x 2 columns]

In [30]:
srsq['price'] = srsq['price'].apply(lambda x: x[-1])  # just using last price
srsq = srsq[['price']]

srsq

price
event section row seat quantity         
1     101     1   1    4         1207.31
                  2    4         1207.31
                  3    4         1207.31
                  4    4         1207.31
                  5    2         1463.50
...                                  ...
      UCTER   2   59   4          222.28
                  60   4          222.28
                  61   4          222.28
                  6292 2         1842.99
                  6293 2         1842.99

[6906 rows x 1 columns]

In [31]:
# from sklearn.linear_model import LinearRegression

# slopes, intercepts = [], []
# for idx, group in srsq.groupby(['section', 'row', 'seat', 'quantity']):
#     # Flatten the lists of values
#     X = np.array([item for sublist in group['days'].values for item in sublist]).reshape(-1, 1)
#     y = np.array([item for sublist in group['price_per_ticket'].values for item in sublist])
    
#     if len(X) > 1:  # Ensure there are enough points for regression
#         model = LinearRegression().fit(X, y)
#         slope = model.coef_[0]
#         intercept = model.intercept_
#     else:
#         slope, intercept = np.nan, np.nan  # Handle insufficient data
    
#     slopes.append(slope)
#     intercepts.append(intercepts)

# srsq['slope'] = slopes
# srsq['intercept'] = intercepts

### Gereralization for quantity

6. We are getting rid of quantity.
7. OPTIONAL. Store our findings for a separate model.

In [32]:
srs = srsq.query('quantity==2') # Removing quantity: Use only quantity equal 2 - the most popular selling option
srs = srs.reset_index(['quantity'])
srs = srs[['price']]
srs

price
event section row seat         
1     101     1   5     1463.50
                  6     1463.50
                  7     1196.35
                  8     1196.35
                  9     1049.76
...                         ...
      UCTER   2   9      171.59
                  12     160.63
                  13     160.63
                  6292  1842.99
                  6293  1842.99

[3857 rows x 1 columns]

### Gereralizing for seats

8. We are getting rid of seat numbers.
9. OPTIONAL. Store our findings for a separate model.

In [33]:
sr = srs.groupby(level=[0, 1, 2]).mean()  # just mean
sr

price
event section row             
1     101     1    1236.536667
              2    1049.760000
              3    1050.445000
              4    1224.206667
              5    1404.590000
...                        ...
      334     15    148.985000
              2W    242.145000
      CRT     1    9383.470000
      UCTER   1     134.600000
              2     725.070000

[706 rows x 1 columns]

### SECOND MAIN OUTPUT: Saving Simplified Dataframe

In [34]:
sr.to_csv('processed2.csv')